In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [ ]:
path = r"E:\data\superstore.xls"

In [ ]:
data = pd.read_excel(path)
data

In [ ]:
data.loc[data['Profit'] <= 0, 'Profitable'] = False
data.loc[data['Profit'] > 0, 'Profitable'] = True
data

Let's create the following columns
1. Order Year
2. Order Day
3. Order Month
4. Profit Margin
    - Profit / Sales
6. Discount ($)
    - Sales * Discount

In [ ]:
data['Order Year'] = data['Order Date'].dt.year
data['Order Day'] = data['Order Date'].dt.strftime('%A')
data['Order Month'] = data['Order Date'].dt.strftime('%B')
data['Profit Margin'] = data['Profit'] / data['Sales']
data['Discount ($)'] = data['Sales'] * data['Discount']

Let's create another column (Discount Bucket), based on a criteria
- for discount in the range 0% - 33% $\implies$ Low
- for discount in the range 33% - 66% $\implies$ Medium
- for discount i the range 66% - 100% $\implies$ High

In [ ]:
data.loc[data['Discount'] < 0.33, 'Discount Bucket'] = 'Low'
data.loc[data['Discount'] >= 0.67, 'Discount Bucket'] = 'High'
data.loc[(data['Discount'] >= 0.33) & (data['Discount'] < 0.67), 'Discount Bucket'] = 'Medium'
data.loc[:, 'Discount Bucket']

In [ ]:
data.loc[data['Discount Bucket'] == 'Medium', ['Discount', 'Discount Bucket']]

### `dataframe_obj.drop(columns=[columns to remove], axis=1)`
- removes columns from the dataframe by default (`axis=1`)
- when `axis=0`, removes specified rows

In [ ]:
# removing columns from a dataframe
data.drop(['Profitable'], axis=1, inplace=True)

### `dataframe_obj.unique()`
- creates an array of unique values in the field specified

In [ ]:
data['Region'].unique()

### `dataframe_obj.sort_values(by=[column_names], ascending=True, inplace=False, axis=0)`
- used to sort values in ascending or descending order.
- sort by ascending order by default; if you want to sort by descending order, specify `ascending=False`
- we can also specify `inplace=True` to update the original dataframe
- can sort columns also, by specifying `axis=1`

In [ ]:
data.sort_values(by=['Order Date'])

In [ ]:
# sorting data by descending order of Profit
data.sort_values(by=['Profit'], ascending=False)

In [ ]:
# sorting multiple fields
data.sort_values(by=['Order Date', 'Category', 'Profit'])

In [ ]:
# specifying different sorting methods for different fields
data.sort_values(by=['Order Date', 'Category', 'Profit'], ascending=[True, True, False])

### `dataframe_obj.value_counts()`
- used to create a simple frequency distribution

In [ ]:
data.value_counts(subset='Customer ID')

In [ ]:
data.value_counts(subset=['Category', 'Sub-Category'])

In [ ]:
type(data.value_counts(subset=['Category', 'Sub-Category']))

In [ ]:
transaction_count = data.value_counts(subset=['Category', 'Sub-Category'])
transaction_count

### `dataframe_obj.reset_index(inplace=False)`
- used to convert index of dataframe into a column and assign default index from 0 to n-1.

In [ ]:
transaction_count = transaction_count.reset_index()
transaction_count

### `dataframe_obj.rename(columns={'old_name':'new_name'}, inplace=False)`
- used to rename headers

In [ ]:
transaction_count.rename(columns={'count':'Transactions'}, inplace=True)
transaction_count

### Performing aggregations

In [ ]:
# sum of sales by category
data.groupby('Category')['Sales'].sum()

In [ ]:
# sum of sales by Category and Sub-Category
data.groupby(['Category', 'Sub-Category'])['Sales'].sum()

In [ ]:
# sum of Sales and Profit by Category and Sub-Category
data.groupby(['Category', 'Sub-Category'])[['Sales', 'Profit']].sum()

### Using `agg()` method to perform aggregations
SYNTAX:
```
dataframe_obj.groupby([columns]).agg({field_1:[aggregation_methods]
                                     ,field_2:[aggregation_methods]
                                     ...
                                     ,field_n:[aggregation_methods]]})
```

In [ ]:
# using the agg() method to perform multiple aggregations
# sum of Sales and average of Sales by Category
data.groupby(['Category']).agg({'Sales':['sum', 'mean']})

In [ ]:
# sum of Sales and Profit by Category and Sub-Category
data.groupby(['Category', 'Sub-Category']).agg({'Sales':'sum'
                                               ,'Profit':'sum'})

In [ ]:
# sum of Sales, sum of Profit and average Discount based on Category and Sub-Category
data.groupby(['Category', 'Sub-Category']).agg({'Sales':'sum'
                                               ,'Profit':'sum'
                                               ,'Discount':'mean'})

### Using a standard syntax for `dataframe_obj.groupby().agg()`
- there are multiple ways to use the `dataframe_obj.groupby().agg()` functionality
- this may make the syntax confusing
- a standard syntax to avoid errors can be
  ```
  dataframe_obj.groupby([columns_to_groupby]).agg({field_1:[aggregation_methods]
                                                  ,field_2:[aggregation_methods]
                                                  ...
                                                  ,field_n:[aggregation_methods]})
  ```

- aggregation method keywords:
  ```
  count
  mean
  sum
  median
  mode
  max
  min
  std
  var
  ```

In [ ]:
# count of transaction by each customer
# previous syntax: data.value_counts(subset='Customer ID')
# standard syntax:
data.groupby(['Customer ID']).agg({'Customer ID':['count']})

In [ ]:
# count of transactions in each sub-category
# previous syntax: data.value_counts(subset=['Category', 'Sub-Category'])
# standard syntax:
data.groupby(['Category', 'Sub-Category']).agg({'Sub-Category':['count']})

In [ ]:
# sum of sales by category
# previous syntax: data.groupby('Category')['Sales'].sum()
# standard syntax:
data.groupby(['Category']).agg({'Sales':['sum']})

In [ ]:
# sum of sales by Category and Sub-Category
# previous syntax: data.groupby(['Category', 'Sub-Category'])['Sales'].sum()
# standard syntax:
data.groupby(['Category', 'Sub-Category']).agg({'Sales':['sum']})

In [ ]:
# sum of Sales and Profit by Category and Sub-Category
# previous syntax: data.groupby(['Category', 'Sub-Category'])[['Sales', 'Profit']].sum()
# standard syntax:
data.groupby(['Category', 'Sub-Category']).agg({'Sales':['sum']
                                               ,'Profit':['sum']})

In [ ]:
data.groupby(['Category', 'Sub-Category']).agg({'Sales':['sum']
                                               ,'Profit':['sum']})

In [ ]:
category_summary = data.groupby(['Category', 'Sub-Category']).agg({'Sales':['sum']
                                                                  ,'Profit':['sum']})

category_summary.reset_index(inplace=True)

In [ ]:
category_summary.columns = category_summary.columns.map('_'.join)

In [ ]:
category_summary

In [ ]:
# calulating average time it takes for a customer to order again
# customer_data
customer_data = data[['Customer ID', 'Order ID','Order Date']].drop_duplicates()
customer_data.sort_values(['Customer ID', 'Order Date', 'Order ID'], inplace=True)
customer_data

In [ ]:
customer_data['Next Order Date'] = customer_data['Order Date'].shift(-1)
for customer in customer_data['Customer ID'].unique():
    customer_data.loc[customer_data['Customer ID'] == customer, 'Next Order Date'] = customer_data.loc[customer_data['Customer ID'] == customer, 'Order Date'].shift(-1)

In [ ]:
customer_data['Duration'] = (customer_data['Next Order Date'] - customer_data['Order Date']).dt.days
customer_data.info()

In [ ]:
customer_data.groupby(['Customer ID']).agg({'Order ID':'count'
                                           ,'Duration':'mean'})

### General Data Clean-Up and Exploratory Data Analysis Guidelines
Steps:
1. ensure all fields are of the correct data type
   - For example, data type of a date column should be date (or similar and not string)
3. get an idea of null/non-null values
    - can be done through `info()` method which tells us the number of non-null values in each column 
4. study each column of the data and get an understanding of values to identify invalid values
    - this step involves standardising data so that we have proper values in the dataset
    - in case of invalid values, we need to treat them as per business specification
    - For example, in Superstore Orders table, for the `Category` field

| Valid | Invalid |
| --- | --- |
| Technology | tech |
| Furniture | Furn. |
| Office Supplies | O. Supplies |
| | Techno |
| | Off. supp. |

5. once data has been sanitized (all columns are of proper data type, all values are standardised), understand the metrics of the data
    - for qualitative fields (dimensions): get the frequency distribution (number of times a value occurs)
    - for quantitative fields (measures): get aggregate summary
This step allows us to understand data points and also identify outliers
    - outlier identification is important as it impacts our analysis
    - we can use describe() method to get summary statistics for various columns in the DataFrame
6. develop understanding of the relation between different pairs of columns and different combination of columns
    - this allows us develop points of analysis
    - for example, if sales in East are high, we should try to understand what is contributing to high sales in East region, is it due to:
        - order frequency
        - long-term customers
        - more purchase of expensive products than inexpensive orders
    - we should also analyse whether sales are uniformally high in all segments in the East region or are sales high only in Corporate segment
7. based on the above analysis, we choose points which are of more importance than other points. These points are arranged and combined to create an end-to-end story about the data. As part of this compilation, we add visualizations in our reports to make it more accessible

### Importing data by sheet name in pandas

By default, when we don't specify a `sheet_name` in the `read_excel()` function, it imports data from the first sheet in the Excel workbook.

We can import a specific sheet by specifying the name of the sheet in the `sheet_name` argument.

In [ ]:
returns = pd.read_excel(path, sheet_name='Returns')
people = pd.read_excel(path, sheet_name='People')

### Listing sheet names in an Excel workbook

In [ ]:
pd.ExcelFile(path).sheet_names

### `dataframe_obj.merge(dataframe_2, on, left_on, right_on, how)`
This method is used to combine two dataframes horizontally based on fields in the dataframe.
- when we don't specify any fields it joins based on common field
- we can combine them using `INNER JOIN` (default), `OUTER`, `CROSS`, `LEFT`, `RIGHT`, which we specify in the `how` parameter
- merge also doesn't require us to rename columns in case there are common columns, it handles duplicate columns on its own

NOTE: in case the key columns have different names, we can specify them:<br>
`left_on` = name of key column in the left table<br>
`right_on` = name of key column in the left table

In [ ]:
pd.merge(data, people)

In [ ]:
pd.merge(data, returns, how='left')

### `dataframe_obj.set_index()`
The `dataframe_obj.set_index(['fields_to_set_as_index'])` is the opposite of `reset_index()` method. It takes in fields which we want to make into the index of the dataframe. This can be useful when we want to make the primary key of our table into an index, instead of keeping it as a column in the dataframe.

In [ ]:
orders = data
orders.set_index(['Order ID'])

In [ ]:
returns.set_index(['Order ID'], inplace=True)

### `dataframe_obj.join(dataframe_2, on, lsuffix, rsuffix, how)`
- this method also combines data horizontally, but based on dataframe indices
- we need to specify the suffix of at least one of the tables in case there are headers with the same name in both the tables
- it performs `LEFT JOIN` by default, but can be changed using the `how` parameter

In [ ]:
orders.join(returns)

### Exporting dataframe to an Excel workbook

In [ ]:
new_file = pd.ExcelWriter(r'E:\data\orders.xlsx')

for sub_category in data['Sub-Category'].unique():
    data[data['Sub-Category'] == sub_category].to_excel(new_file, sheet_name=sub_category, index=False)

new_file.close()

### `pd.concat([dataframes], axis)`
- this method is used to combine multiple dataframes into one
- either vertically (default) or horizontally (axis=1)
- usually used for `UNION`ing tables
- the concat() method keeps duplicate rows, these can be removed using the drop_duplicates() method

### Importing multiple Excel sheets into one dataframe

In [ ]:
workbook_path = r'E:\data\orders.xlsx'

sub_category_sheets = pd.ExcelFile(workbook_path).sheet_names
sub_category_sheets

In [ ]:
sub_category_data = []
for sheet in sub_category_sheets:
    sub_category_data.append(pd.read_excel(workbook_path, sheet_name=sheet))

pd.concat(sub_category_data)   

In [ ]:
orders_data = pd.concat(sub_category_data)
orders_data.reset_index(inplace=True)

In [ ]:
orders_data.drop(columns=['index'], inplace=True)
orders_data

### Importing data from multiple CSV files in a folder into a dataframe

In [ ]:
import os

folder_path = r'E:\data\stocks_data'

os.listdir(folder_path)

In [ ]:
stocks_data = []
for file in os.listdir(folder_path):
    stocks_data.append(pd.read_csv(folder_path + '\\' + file))

consolidated_data = pd.concat(stocks_data)

In [ ]:
consolidated_data.shape

### Checking for duplicate rows

When we are dealing with data dynamically, we should not hard-code any values. One method of fetching all the rows that are duplicate in the dataset can be as follows:

In [ ]:
# Step 1. place all headers in the table in the groupby method and get the count of any column in the table
consolidated_data.groupby(list(consolidated_data.columns)).agg({consolidated_data.columns[0]:'count'})

In [ ]:
# storing the above table into a new variable
duplicate_check = consolidated_data.groupby(list(consolidated_data.columns)).agg({consolidated_data.columns[0]:'count'})

In [ ]:
duplicate_check.columns

In [ ]:
# Step 2. storing the name of the column which contains count of duplicates into a variable
count_header = duplicate_check.columns[0]

In [ ]:
# Step 3. fetching only those rows in the duplicate_check where the count is greater than 1
duplicate_check[duplicate_check[count_header] > 1]

In [ ]:
# Step 4. storing above data into a new variable after renaming the the field containing count to 'count'
duplicate_rows = duplicate_check[duplicate_check[count_header] > 1].rename(columns={count_header:'count'})
duplicate_rows

In [ ]:
# Step 5. converting columns back into data fields from index
duplicate_rows.reset_index(inplace=True)
duplicate_rows

In [ ]:
# Step 6. deleting the count column
duplicate_rows.drop(columns=['count'], inplace=True)
duplicate_rows

### Downloading a file from a URL

In [ ]:
from urllib.request import urlretrieve

specifying URL of the file
url = r'https://raw.githubusercontent.com/puneettrainer/datasets/main/insurance_fraud.csv'

extracting file name from the URL
filename = url.split('/')[-1]

downloading the file and saving it under the file name extracted
urlretrieve(url, filename)